# PURPOSE:

#### This is a jupyter notebook that will orchestrate the data processing notebook scripts that will perform the following:

1. check the before and after CPUs and then
2. generate the raw "Red and Green sheet" data
3. Creates the summarized "Red and Green Sheet" in native currency
4. Creates the summarized "Red and Green Sheet" in USD

Orchestration of notebooks are handled by a library called [papermill](https://github.com/nteract/papermill).  To pass data between notebooks, the [scrapbook](https://github.com/nteract/scrapbook/) library is being used.

In [ ]:
from win10toast import ToastNotifier
import papermill as pm
import time

In [ ]:
start_time = time.time()

# Enter necessary parameters:

### Enter parameter for CLA claim month or RVMS reporting month:

In [ ]:
CLA_MONTH = input("Enter CLA Claim Month ('YYYYMM'): ")

### Enter start and ending model years that are being adjusted or reported on:

In [ ]:
start_year = int(input("Enter starting model year (YYYY): "))
end_year = int(input("Enter ending model year (YYYY): "))

### For the summarized Red and Green Sheet, enter KC to USD exchange rate:

In [ ]:
exchange_rate = float(input("Enter KC to USD exhange rate: "))

## 1) Execute jupyter notebook that obtains the current budgeted CPUs in the the RVMS database server:

In [ ]:
%%time
pm.execute_notebook(
    '1_Obtaining_Current_RVMS_GraphCat_Group_SubGroup_Budgeted_CPUs-Access.ipynb',
    'output/CurrentBudgetedCPUs.ipynb',
    cwd='D:\\jupyter\\rvms\\production',
    parameters = dict(CLA_MONTH=CLA_MONTH)
)

## 2) Execute jupyter notebook that performs the before and after CPU checks, creates CPU differences columns, and creates the raw Red and Green sheet data:

In [ ]:
%%time
pm.execute_notebook(
    '2_Comparing_Adjusted_GraphCat_CPUs_against_Original_GraphCat_CPUs-Access.ipynb',
    'output/RedAndGreenSheet.ipynb',
    cwd='D:\\jupyter\\rvms\\production',
    parameters = dict(CLA_MONTH=CLA_MONTH)
)

## 3) Execute jupyter notebook that summarizes the raw "Red and Green Sheet" data in native currency:

In [ ]:
%%time
pm.execute_notebook(
    '3_Get_Top3_Bottom3_Group_SubGroup_Adjustments_Native_Currency.ipynb',
    'output/RedAndGreenSheet_Summarized_Native.ipynb',
    cwd='D:\\jupyter\\rvms\\production',
    parameters = dict(start_year=start_year,
                      end_year=end_year
                     )
)

## 4) Execute jupyter notebook that summarizes the raw "Red and Green Sheet" data in US dollars:

In [ ]:
%%time
pm.execute_notebook(
    '4_Get_Top3_Bottom3_Group_SubGroup_Adjustments-USD.ipynb',
    'output/RedAndGreenSheet_Summarized_USD.ipynb',
    cwd='D:\\jupyter\\rvms\\production',
    parameters = dict(exchange_rate=exchange_rate,
                      start_year=start_year,
                      end_year=end_year,
                      CLA_MONTH=CLA_MONTH
                     )
)

In [ ]:
elapsed_time = time.time() - start_time

# elapsed_time is in seconds, but want to define seconds as hour, minute, and seconds
m, s = divmod(elapsed_time, 60)
h, m = divmod(m, 60)
f"{h:1.0f}:{m:1.0f}:{s:.2f}"

### If the script made it this far, then the scripts successfully executed without errors.  Then send Windows 10 toast notification.

In [ ]:
toaster = ToastNotifier()
toaster.show_toast("Papermill Orchestration Status",
                   "Successfully executed all ETL scripts",
                   icon_path="images/honda_logo.ico",
                   duration=5)